<a href="https://colab.research.google.com/github/AnuragRaj11/IIT-KGP-Hackathon/blob/main/Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report, mean_squared_error, r2_score)
from imblearn.over_sampling import SMOTE
import shap

In [7]:
df = pd.read_excel("dataset_excavate.xlsx")


In [9]:
print(df.columns.tolist())

['functional group', 'A', 'A_OS', "A'", "A'_OS", 'A_HOMO-', 'A_HOMO+', 'A_IE-', 'A_IE+', 'A_LUMO-', 'A_LUMO+', 'A_X-', 'A_X+', 'A_Z_radii-', 'A_Z_radii+', 'A_e_affin-', 'A_e_affin+', 'Bi', 'B_OS', "B'", "B'_OS", 'B_HOMO-', 'B_HOMO+', 'B_IE-', 'B_IE+', 'B_LUMO-', 'B_LUMO+', 'B_X-', 'B_X+', 'B_Z_radii-', 'B_Z_radii+', 'B_e_affin-', 'B_e_affin+', 'PBE band gap', 'μ', 'μĀ', 'μ𝐵\xa0̅', 't']


In [11]:
df['insulator'] = (df['PBE band gap'] >= 0.5).astype(int)

In [12]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
features = [col for col in num_cols if col not in ['PBE band gap', 'insulator'] ]

In [13]:
X_class = df[features]
y_class = df['insulator']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)